In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
from features.settings import Baseline_Settings, Triplet_Hinge_Settings, Triplet_Log_Settings, Pair_Hinge_Settings, Triplet_Distance_Settings
from features.models import GCNNSmall0, GCNNMedium0, GCNNMedium1, GCNNLarge0
from learn import get_feature_model, train_features, test_features
from datasets import load_cifar10

/home/dqi/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/dqi/.conda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data, samplers, pair_samplers, triplet_samplers = load_cifar10(augment=True)
d=data; s=triplet_samplers

CIFAR-10 loaded in 105 seconds


# Train features

In [3]:
tf.reset_default_graph()
f, f_params = get_feature_model(feature_model=GCNNLarge0, settings=Triplet_Distance_Settings,
                                num_features=10, data_format=[32, 32, 3],
                                f_lr=0.03, f_reg=0.001, f_scope="d_gl0_cifar10_10")
fs = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "d_gl0_cifar10_10"))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_features(sess=sess, feature_model=f, samplers=s, num_steps=40000, keep_prob=0.7, f_params=f_params)
    fs.save(sess, "./models/cifar10/d_gl0_cifar10_10.ckpt")

begin training features
	step 0: train loss 4.06642, train error 0.453333, valid loss 4.33187, valid error 0.453333
	step 200: train loss 1.46553, train error 0.436667, valid loss 1.46108, valid error 0.41
	step 400: train loss 1.3643, train error 0.396667, valid loss 1.40989, valid error 0.416667
	step 600: train loss 1.21427, train error 0.326667, valid loss 1.22942, valid error 0.396667
	step 800: train loss 1.28561, train error 0.35, valid loss 1.36646, valid error 0.453333
	step 1000: train loss 1.29299, train error 0.403333, valid loss 1.22002, valid error 0.353333
	step 1200: train loss 1.15474, train error 0.31, valid loss 1.13809, valid error 0.266667
	step 1400: train loss 0.97503, train error 0.26, valid loss 1.01526, valid error 0.29
	step 1600: train loss 1.08092, train error 0.31, valid loss 1.06107, valid error 0.303333
	step 1800: train loss 1.03293, train error 0.3, valid loss 0.996331, valid error 0.27
	step 2000: train loss 0.954867, train error 0.276667, valid loss 

# Test features

In [3]:
tf.reset_default_graph()
f, f_params = get_feature_model(feature_model=GCNNLarge0, settings=Triplet_Distance_Settings,
                                num_features=10, data_format=[32, 32, 3],
                                f_lr=0.03, f_reg=0.001, f_scope="d_gl0_cifar10_10")
fs = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "d_gl0_cifar10_10"))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    fs.restore(sess, "./models/cifar10/d_gl0_cifar10_10.ckpt")
    test_features(sess=sess, feature_model=f, samplers=s, f_params=f_params)

INFO:tensorflow:Restoring parameters from ./models/cifar10/d_gl0_cifar10_10.ckpt
test set error: 0.0686 // time elapsed: 177.4232 s
